# ChemFlow Docking tutorial - Part 2: Virtual screening to predict binding poses 
## Analysis of DockFlow results, introducing <b>Report<i>Flow</i><b>.

* Learning goals
    * Reading and manipulating rank.csv as a DataFrame.
    * 


### Fill up the configuration lines bellow and have fun with jupyter-notebook.

In [ ]:
# Configuration
project="Myosin6"   # Name for your project
protocol="Blebbistatin"   # If you didn't put any name, it should be "default"
target="receptor"        # Name of the receptor.

In [ ]:
# Import the required python modules. Please don't change anytyhing here.
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

## Read the data.
Dock<i>Flow</i> --postdock produced the <b>rank.csv</b> and <b>docked_ligands.mol2</b>, gathering all docking poses from all compounds for a certain receptor within a protocol into <b>docked_ligands.mol2</b> and their information into <b>rank.csv</b>, including protocol name, docking program, ligand, pose and energy, all in Tidy Data format making it <i>as easy as pie</i> to manipulate. 

### The following line will read <b>rank.csv</b> data into a <b>DataFrame</b>, using the <b>pandas</b> module.

In [ ]:
# Read 'rank.csv' and save to a DataFrame. 
# It contains values separated by "spaces", we need to define "delim_whitespace=True".
df=pd.read_csv('/home/gpereira/{}.chemflow/DockFlow/{}/{}/rank.csv'.format(project,protocol,target),delim_whitespace=True)

### Review the data.
Let's see the first 5 lines of this dataframe, if you want, remove ".head(5)" to see it all.
It's always nice to see if we loaded things properly.  
As you can see, the dataframe has the raw contents of your file.  

Notice, since "rank.csv" is a .CSV file. The first line (header) contains column names, which will be automatically assigned.

In [ ]:
# Let's see the first 5 lines of this dataframe. You may remove " .head(5) " or change the number of lines you want to see.
df.head(10)

#### Get a glance of the energy distribution.
Below we present the <b>Scoring distribution</b> both as a histogram and a kde plot. We're using the python "seaborn" package.

Blebbistatin redock energy range from: -104.5 to -63.2 for redock. Curiously it was a little different for "rescoring" the crystal structure: -100.16 

The graph below shows the distribution of SCORES from all CN docking poses, with a vertical line indicating <b>blebbistatin</b> SCORE from a simple rescore.



In [ ]:
# Seeing the Scoring distribution both as a histogram and a kde plot.
# We're using the python "seaborn" package.
sns.distplot(df['SCORE'])

# You may add graphical elements to assist your decisions during graph analysis.
# Add a vertical line at BLEBBISTATIN SCORE from redocking.
plt.plot([0, -100.16], [1.5, 0], linewidth=1)

# Defines a limit for the graph.
plt.xlim(df['SCORE'].min(),0)

plt.savefig('ScoreDist_CN.pdf')

In [ ]:
#How many ligands are there with energy below blebbistatin?
tmp=df.groupby('LIGAND').head(1)
tmp[tmp['SCORE'] < -100.16 ].count()

In [ ]:
#comps = pd.read_table('/home/gpereira/knime-workspace/GIlberto_CN_Project/CN_desalted_lipinski_PAINS_DWAR_flexo_clean_graphs.txt')

# Property list
list=['mol.id','Druglikeness','cLogP','cLogS','AMW']

# Read the table but only the listed properties
comps = pd.read_table('/home/gpereira/knime-workspace/GIlberto_CN_Project/CN_desalted_lipinski_PAINS_Isomers_Tautomers_3D.txt',usecols=list)

# Group data according to "mol.id" into "Tidy-data" format, required for a good SEABORN graph.
comps2 = comps.melt('mol.id',var_name='Property', value_name='Value')

g = sns.FacetGrid(comps2, col="Property",sharex=False,sharey=False)
g = (g.map(sns.distplot, "Value"))
plt.show()

g.savefig('Properties_CN.pdf')

## The sigmoid curve.
It is a common practice to see the distribution as a sigmoid curve. Together with the previous distribution it helps to define a cutoff energy to progress.
doi:10.1021/jm301916b

Check how the original distribuion behaves, then adjust the cutoff however it pleases you.

In [ ]:
# Set here the cutoff.
cutoff=-104.519

# Sort the Values and reset index for plotting.
df2=df.sort_values('SCORE')
df2.reset_index(inplace=True)
df2['SCORE'].plot()
plt.savefig('EnergyDist_OverNumber1.pdf')
plt.show()


# Same plot as above, now with a maximun limit to Y-axis.
df2['SCORE'].plot()
plt.ylim(df2['SCORE'].min(),0)
plt.show()

# The distribution applying "cutoff".
df3 = df2[df2['SCORE'] < cutoff ]
df3['SCORE'].plot()
plt.ylim(df3['SCORE'].min(),0)

plt.savefig('EnergyDist_OverNumber.pdf')

## CHALLENGE #1 - Counts and plots
* How many compounds with energy better than -90 "score" ?  

* Show these same graphs only for those compounds.

* The scoring filter was applyed to "SCORE". You may now have less then 10 poses/ligand.


In [ ]:
# Use the same cutoff as abouve, or set a new one.
result1 = df3[ df3['SCORE'] < -100 ]
print(result1.count())

#Object containing total number of unique ligands
ASD = result1.groupby('LIGAND').head(1)

#Counts number of unique ligands
ASD.count()

#Calculates the average of 'SCORE' of the Ligands
ASD.mean()

ASD.max()

ASD.min()
del result1['index']


df2=df.sort_values('SCORE')
df2.reset_index(inplace=True)
df2['SCORE'].plot()
plt.show()

results4=df2.reset_index(inplace=True)

sns.distplot(results4['SCORE'])
plt.show()

In [ ]:

result1['SCORE'].plot(kind='kde')
plt.show()
result1['SCORE'].plot(kind='box')
plt.show()
result1['SCORE'].plot(kind='hist')
plt.show()

sns.distplot(result1['SCORE'])
plt.show()

plt.savefig('ScoreDist_RefBle.pdf')


#### Work a little with the dataframe.
Depending on the docking program the output name could be different. For PLANTS, it writes an <b>entry</b> number followed by the docking <b>pose</b>.
The following lines exemplify how to manipulate of the DataFrame in order to extract the pose number from a string in the POSE column.

In [ ]:
# create a temporary DataFrame splitting the text from "POSE" column.
POSE_NUMBER=df['POSE'].str.split('_conf_',expand=True)

# Then assign a new column to "df" Dataframe, using the contents of the second column of POSE_NUMBER.
# Numbering in python arrays start with number "0", so position "2" is actually referenced by number "1".
df['POSE_NUMBER']=POSE_NUMBER[1]

# See the first 5 lines
df.head(5)

#### Produce a quick description of the dataframe.
You can use <b>describe</b> to get a glance at the statistics of your DataFrame.  
<b>Pandas</b> is smart enough to find which columns contain "number" to do the Statistics.  
To get LIGAND-based data, you can <b>group</b> by ligand.

In [ ]:
df.groupby('LIGAND').describe().head(10)

#### Let's do some visualization ! Display how the docking poses distribute.
Below plot the Scoring distribution by docking pose. (may take 2-3 minutes)
Try other visualizations !

In [ ]:
sns.swarmplot(data=df,y='SCORE',x='POSE_NUMBER')
plt.show()

g = sns.FacetGrid(df, col="POSE_NUMBER", col_wrap=5)
g = g.map(sns.distplot, "SCORE")


# To show a second graph, you must first "show" the one above.
plt.show()

# Violin plots are "trendy" nowaday. It's a cool way to show the same graph.
sns.violinplot(data=df, y="SCORE", x="POSE_NUMBER")

In [ ]:

# Do it here and show us the graphs.
# 

### Getting into details
So far we have a panorama of the energies, to get into details we must narrow the ligands we want to see.

Follow slowly the commands below to MASTER the art of "pandas" data manipulations.

#### How about listing of BEST docking pose per ligand ? It should be "${LIGAND}_conf_01"

In [ ]:
compounds=df.groupby('LIGAND').min()
compounds.head(10)

#### Same as above, now sorted by SCORE. # Notice it's a quite different approach from above.

In [ ]:
compounds=df.sort_values('SCORE').groupby('LIGAND').head(1)
compounds.head(10)

#### Ever better, show me only the 2 top poses of each ligand.

In [ ]:
compounds=df.sort_values('SCORE').groupby('LIGAND').head(2)

# Display the top 5 only.
compounds.head(10)

#### As you can see the it's now sorted by "SCORE". To recover the original order "LIGAND" you can just resort based on the original index using ".sort_index()".


In [ ]:
compounds=df.sort_values('SCORE').groupby('LIGAND').head(2).sort_index()
compounds.head(10)

#### Not sorting at all, just show me the first 2 poses for each ligand.

In [ ]:
# Same as above, now sorted by energy.
# Notice it's a quite different approach.
compounds=df.groupby('LIGAND').head(2)
compounds.head(5)

# CHALLENGE #2 - Can you manipulate the data ?
Show the distribuition only the 2 first docking POSE of each ligand ?

In [ ]:
# Sort the list, group by ligand then give me the best two from each ligand.
compounds2 = df3.sort_values('SCORE').groupby('LIGAND').head(2).sort_index()

# Create a list of values I'd like to use.
lista = ['LIGAND', 'POSE', 'SCORE','POSE_NUMBER']
 
# Get only the properties from "lista"
compounds2 = compounds2[lista]

# POSE_NUMBER so far is a "string". Convert it to a "number", using "numpy"
# Now, reshape the dataframe to "Tidy-format" using "LIGAND" and "POSE_NUMBER" as reference, put all other properties into "Property"
result = compounds2.melt(['LIGAND','POSE','POSE_NUMBER'], var_name='Property', value_name='Value')
result

# Make a cool graph, coloring by pose number! 
g = sns.FacetGrid(result, col="Property",sharex=False,sharey=False)
g = (g.map(sns.distplot, "Value"))
plt.show()


#g = sns.FacetGrid(result, col="Property")
#g = (g.map(sns.boxplot, "Value"))
#plt.show()


### Back in business. 
How is the score among the docking poses of the TOP 50 compounds ?
A summarized way to see that is with a BOXPLOT.

Notice how spread some of the SCORES are for each ligand !

In [ ]:
# Shows the 5 first entries for each data set ligand in a dataframe sorted by score, grouped by ligand. (maximum number of entries for each ligand)

compounds=df.sort_values('SCORE').groupby('LIGAND').head(5)

# Lists the x first entries of the created dataframe. For each ligand, more than one pose may be represented
#compounds.head(500)

# Plots the first 10 entries of the data set: x axis are ligands, y axis are score values
sns.boxplot(data=compounds.head(10),x="LIGAND",y='SCORE')

### Same graph, now you can actually read the labels.
Sometimes it will be necessary to manipulate the graph to correct the imperfections. The example below repeats the graph above, now with rotated <b>xaxis</b> tickmarks.

In [ ]:
ax = sns.boxplot(data=compounds.head(50),x="LIGAND",y='SCORE')
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

## CHALLENGE #3 - Did you notice results are NOT sorted by the best SCORE ? 
Also, the plot is showing the top 50 compounds <b>from the list</b> and those are not even 50 different molecules! 
Work out the data: sort by SCORE and replot the graph above with ONLY the TWO best poses/ligand.

Finally, produce a list of top scoring molecules to use in the next step. (you may use "ChemFlow_tools" for that)

In [ ]:
# Step 1 - Create a list of compounds, sorted by "SCORE", containing the two best docking poses.
compounds=df.sort_values('SCORE').groupby('LIGAND').head(1)


# Step 2 - Creates a second list, using the dataframe entries (10) sorted by "SCORE". Only the best docking pose is written in the list
best_compounds=df.sort_values('SCORE').groupby('LIGAND').head(1).head(2500)


# usefull command: compounds.isin(compound_list)


# Step 3 -  Compares the two lists created by their "LIGAND" entry and, if a match is found, prints the two best docking poses for the (10) ligands.
results_table = compounds[compounds['LIGAND'].isin(best_compounds['LIGAND'])]

#compounds[compounds['LIGAND']=="13071-11-9"] - Access to all docking poses obtained for a given compounds within the "compounds" dataframe


In [ ]:
# Step 1 - Create a list of compounds, sorted by "SCORE", containing the two best docking poses.
compounds=df.sort_values('SCORE').groupby('LIGAND').head(1)
compounds

In [ ]:
# sets figure size (x,y)
plt.figure(figsize=(10,20))

# faz um plot
ax = sns.barplot(data=results_table,x='SCORE',y='LIGAND',hue='POSE_NUMBER')

# Labels rotate 90 degrees
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

ax.set_title('Docking score by pose')
ax.set_xlim(results_table['SCORE'].min(),-104.519)
#ax.figsize=(20,30)


# Write the results to a csv or xls file.
Your last results are named "results_table" so lets save them to a .csv file using <b>pandas</b>

In [ ]:
# Save a dataframe to a .csv file.
results_table.to_csv('/home/gpereira/blebb_table.csv',index=False)
results_table.to_excel('/home/gpereira/blebb_table.xls')

# Finally, save a figure to a .png or .pdf file.
Repeat the last figure, but now, <b>before </b> plotting, save the figure.

In [ ]:
# sets figure size (x,y)
plt.figure(figsize=(10,20))

# faz um plot
ax = sns.barplot(data=results_table,x="SCORE",y='LIGAND',hue='POSE_NUMBER')

# Labels rotate 90 degrees
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

ax.set_title('Docking score by pose')
ax.set_xlabel('Docking Score')
ax.set_xlim(results_table['SCORE'].min(),-104.519)
#ax.figsize=(20,30)

# Sometimes some areas of the graph are outside the figure.
# A quick fix is to use the "Tight layout"
plt.tight_layout()

# Save the figure 
plt.savefig('figure.pdf')

#
plt.savefig('figure.png',dpi=300)


In [ ]:
# sets figure size (x,y)
plt.figure(figsize=(5,10))

# faz um plot
ax = sns.distplot(results_table["SCORE"])

# Labels rotate 90 degrees
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

ax.set_title('Docking Score')
#ax.set_xlabel('Docking Score')
ax.set_xlim(results_table[SCORE].min())
ax.figsize=(20,30)

# CHALLENGE - Histogram plot

# Interaction FingerPrints
Run ChemFlow_tools to extract the interaction fingerprints compared to blebbistatin (as tanimoto distance), the output should be <b>IFP.dat</b>.
* Read it and show the similarity distribution with the graphs you've learned above
* Select a range of IFP (the higher the better) and selected ~10 <b>different</b> ligands with the highest similarity. Those will pass to ScoreFlow. 

In [ ]:
IFPs=pd.read_csv('/home/gpereira/IFP.tanimoto', header=None, delim_whitespace=True)

# removes first row of table. It was the IFP of blebbistatin vs blebbistatin (redundancy).
IFPs.drop([0],inplace=True)

# Renames the columns
IFPs.columns=['Reference','Compound','IFP similarity']
#plots the distribution of IFPs using a histogram plot
sns.distplot(IFPs['IFP similarity'])
plt.show()

#sns.distplot(IFPs['IFP similarity'][IFPs['IFP similarity'] > 0.5])

IFPs[IFPs['IFP similarity'] > 0.5].count()

# You're done for now. Back to Google Docs !